# Dip buy test

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as s
from IPython.display import clear_output


In [8]:
df = pd.read_csv("../data/tpsl-test.csv")
df

,target,close,high,low
0,0.0,38300.300781,38363.449219,38282.781250
1,0.0,38259.359375,38317.191406,38250.859375
2,0.0,38203.210938,38261.128906,38203.199219
3,0.0,38138.328125,38203.199219,38126.000000
4,0.0,38044.160156,38142.039062,37953.441406
...,...,...,...,...
199995,0.0,38682.519531,38689.558594,38655.910156
199996,0.0,38649.699219,38682.531250,38643.070312
199997,0.0,38682.519531,38682.519531,38649.691406
199998,0.0,38649.378906,38682.691406,38640.398438


In [3]:
df = df[0:100000]

In [24]:
def strategy(tp_pct, sl_pct):
    """for optimizing paramaters"""
    btc = None
    price = None
    tp = None
    sl = None

    total = 100
    total_and_btc = None
    amt = 20

    plot = [] #[None, ]

    for i in range(len(df)):
        close = df.iloc[i].close
        high = df.iloc[i].high
        low = df.iloc[i].low
        target = df.iloc[i].target
        if sl and low<sl: #Stop loss hit, sell
            plot.append(-sl)
            total += btc*close*0.999
            btc = None
            price = None
            tp = None
            sl = None
        elif tp and high>tp: #take profit hit, sell
            plot.append(-tp)
            total += btc*close*0.999
            btc = None
            price = None
            tp = None
            sl = None
        elif not tp and not sl and target: #Target says to buy and can buy:
            btc = amt*0.999/close
            price = close
            tp = (1+tp_pct)*close
            sl = (1-sl_pct)*close
            total -= amt
            plot.append(price)
        else:
            plot.append(None)
        b = amt if btc != None else 0
        total_and_btc = total + b
    return total_and_btc

In [12]:
# strategy(0.01, 0.01)

In [26]:
d = {}

In [27]:
for i1 in [0.01, 0.015, 0.02, 0.025]:
    for i2 in [0.01, 0.015, 0.02, 0.025]:
        d["tp:" + str(i1) + " sl:" + str(i2)] = strategy(i1, i2)
        clear_output()
        print(max(d.values()))
        print(d)

110.89922823461013
{'tp:0.005 sl:0.005': 82.7258982043383, 'tp:0.005 sl:0.01': 94.12930426264066, 'tp:0.005 sl:0.015': 98.87110368730481, 'tp:0.005 sl:0.02': 100.38052097547262, 'tp:0.005 sl:0.025': 100.39322146911371, 'tp:0.005 sl:0.03': 98.91699529005012, 'tp:0.01 sl:0.005': 94.48352353730075, 'tp:0.01 sl:0.01': 103.29456440518672, 'tp:0.01 sl:0.015': 105.02724370373645, 'tp:0.01 sl:0.02': 107.15600332890965, 'tp:0.01 sl:0.025': 107.27000820551228, 'tp:0.01 sl:0.03': 104.463222427411, 'tp:0.015 sl:0.005': 97.8499501131605, 'tp:0.015 sl:0.01': 106.39137455572356, 'tp:0.015 sl:0.015': 105.25452038635139, 'tp:0.015 sl:0.02': 109.6234009783987, 'tp:0.015 sl:0.025': 110.89922823461013, 'tp:0.015 sl:0.03': 108.63217764635515, 'tp:0.02 sl:0.005': 99.81148212606732, 'tp:0.02 sl:0.01': 105.072785107097, 'tp:0.02 sl:0.015': 105.5444375890566, 'tp:0.02 sl:0.02': 104.39568088103645, 'tp:0.02 sl:0.025': 107.6952614721099, 'tp:0.02 sl:0.03': 103.58120155354257, 'tp:0.025 sl:0.005': 98.211625139941

In [ ]:
tp_pct = 0.07
sl_pct = 0.05

btc = None
price = None
tp = None
sl = None

buy = False

total = 100
total_and_btc = None
total_plot = []
amt = 20

# cooldown = 0

plot = [] #[None, ]

for i in range(len(df)):
#     if cooldown > 0:
#         cooldown -= 1
#         plot.append(None)
#         continue
    close = df.iloc[i].close
    high = df.iloc[i].high
    low = df.iloc[i].low
    target = df.iloc[i].target

    if sl and low<sl: #Stop loss hit, sell
        plot.append(-sl)
        total += btc*close*0.999
        btc = None
        price = None
        tp = None
        sl = None
#         cooldown += 600
        
    elif tp and high>tp: #take profit hit, sell
        plot.append(-tp)
        total += btc*close*0.999
        btc = None
        price = None
        tp = None
        sl = None
#         buy=True
    elif not tp and not sl and (target or buy): #Target says to buy and can buy:
        btc = amt*0.999/close
        price = close
        tp = (1+tp_pct)*close
        sl = (1-sl_pct)*close
        total -= amt
        plot.append(price)
        buy = False
    else:
        plot.append(None)
    b = amt if btc != None else 0
    total_and_btc = total + b
    total_plot.append(total_and_btc)

In [ ]:
a = 0
b = 200000
fig,ax = plt.subplots(figsize=(20, 10))



plot_buy = list(map(lambda x: None if x == None or x<0 else x, plot[a:b]))
plot_sell = list(map(lambda x: None if x == None or x>0 else -x, plot[a:b]))

# ax.plot(acc[a:b])

# ax1 = ax.twiny()

ax.scatter(range(b-a), 
            plot_buy,
            color="#43aa8b")

ax.scatter(range(b-a), 
            plot_sell,
            color="red")

ax.set_xlabel("time")
ax.set_ylabel("close")
ax.plot(list(df.close)[a:b],
        color="gray")
# ax.plot(list(df.high)[a:b],
#         color="lightgreen")
# ax.plot(list(df.low)[a:b],
#         color="pink")

ax1 = ax.twinx()
ax1.plot(total_plot[a:b])
# ax1.plot(usd[a:b])
# ax.plot(blimit[a:b], color="lightgray")
# ax.plot(slimit[a:b], color="lightgray")

ax.tick_params(labelbottom=False)
ax.axes.xaxis.set_visible(False)